In [1]:
import pandas as pd

In [2]:
pref = pd.read_csv("COVID-19-jp/nhk_news_covid19_prefectures_daily_data.csv")
pref["日付"] = pd.to_datetime(pref["日付"] )
print(len(pref))
pref.tail()

23594


,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計
23589,2021-05-27,47,沖縄県,240,15983,0,148
23590,2021-05-28,47,沖縄県,313,16296,0,148
23591,2021-05-29,47,沖縄県,335,16631,0,148
23592,2021-05-30,47,沖縄県,271,16902,0,148
23593,2021-05-31,47,沖縄県,142,17044,2,150


In [3]:
latest_date = pref["日付"].max().strftime("%Y-%m-%d")

In [4]:
all_jp = pref.groupby(["都道府県名"], as_index=False).sum()
del all_jp["各地の感染者数_累計"]
del all_jp["各地の死者数_累計"]
#all_jp = all_jp.rename(columns={"各地の感染者数合計":"各地の感染者数_1日ごとの発表数"})
all_jp = all_jp.rename(columns={"各地の感染者数_1日ごとの発表数":"各地の感染者数合計"})
all_jp = all_jp.rename(columns={"各地の死者数_1日ごとの発表数":"各地の死者数合計"})
all_jp.tail()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計
42,静岡県,11044,8390,137
43,香川県,18574,2002,24
44,高知県,19578,1468,20
45,鳥取県,15562,465,2
46,鹿児島県,23092,3327,30


## 都道府県ごとの最新データを付与

In [5]:
# 直近１週間の感染者数
pref["直近1週間の感染者数"] = pref["各地の感染者数_1日ごとの発表数"].rolling(7).sum()
pref.tail(7)

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数
23587,2021-05-25,47,沖縄県,256,15441,0,148,1355.0
23588,2021-05-26,47,沖縄県,302,15743,0,148,1454.0
23589,2021-05-27,47,沖縄県,240,15983,0,148,1496.0
23590,2021-05-28,47,沖縄県,313,16296,0,148,1602.0
23591,2021-05-29,47,沖縄県,335,16631,0,148,1706.0
23592,2021-05-30,47,沖縄県,271,16902,0,148,1821.0
23593,2021-05-31,47,沖縄県,142,17044,2,150,1859.0


### 感染者数前日比

In [6]:
import numpy as np
pref["新規感染者数前日比"] = pref["各地の感染者数_1日ごとの発表数"].pct_change()
# 一番古い日付だけNANを設定
pref.loc[pref["日付"]==pref["日付"].min(),"新規感染者数前日比"] = np.nan
pref["直近1週間の新規感染者数前日比平均"] = pref["新規感染者数前日比"].rolling(7).mean()
pref.tail()

,日付,都道府県コード,都道府県名,各地の感染者数_1日ごとの発表数,各地の感染者数_累計,各地の死者数_1日ごとの発表数,各地の死者数_累計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均
23589,2021-05-27,47,沖縄県,240,15983,0,148,1496.0,-0.205298,0.134188
23590,2021-05-28,47,沖縄県,313,16296,0,148,1602.0,0.304167,0.171147
23591,2021-05-29,47,沖縄県,335,16631,0,148,1706.0,0.070288,0.164625
23592,2021-05-30,47,沖縄県,271,16902,0,148,1821.0,-0.191045,0.183715
23593,2021-05-31,47,沖縄県,142,17044,2,150,1859.0,-0.476015,0.163332


In [7]:
latest = pref.loc[pref["日付"]==pref["日付"].max()][["都道府県名","直近1週間の感染者数","新規感染者数前日比","直近1週間の新規感染者数前日比平均"]]
latest.head()

,都道府県名,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均
501,北海道,3012.0,-0.031250,-0.014804
1003,青森県,174.0,10.000000,1.430487
1505,岩手県,90.0,-0.333333,0.041044
2007,宮城県,117.0,-0.600000,0.034394
2509,秋田県,13.0,NaN,NaN


In [8]:
all_jp = pd.merge(all_jp, latest, on="都道府県名")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均
0,三重県,12048,4913,106,198.0,0.157895,0.301794
1,京都府,13052,15823,212,374.0,-0.666667,0.046542
2,佐賀県,20582,2503,19,81.0,-0.500000,0.064085
3,兵庫県,14056,39705,1181,761.0,-0.712871,-0.039807
4,北海道,502,37959,1112,3012.0,-0.031250,-0.014804


## 人口データを付与

In [9]:
pop = pd.read_excel("COVID-19-jp/2001stjin.xls",header=3)[["Unnamed: 1","計"]]
pop.rename(columns={"Unnamed: 1":"都道府県名"},inplace=True)
pop.rename(columns={"計":"人口"},inplace=True)
pop = pop.loc[pop["都道府県名"]!="合計"].reset_index(drop=True)

In [10]:
all_jp = pd.merge(all_jp, pop, on="都道府県名", how="left")
all_jp.head()

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,人口
0,三重県,12048,4913,106,198.0,0.157895,0.301794,1813859
1,京都府,13052,15823,212,374.0,-0.666667,0.046542,2545899
2,佐賀県,20582,2503,19,81.0,-0.500000,0.064085,823810
3,兵庫県,14056,39705,1181,761.0,-0.712871,-0.039807,5549568
4,北海道,502,37959,1112,3012.0,-0.031250,-0.014804,5267762


## 人口当たりの感染者数

In [11]:
all_jp["１万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 10000)
all_jp["10万人当たりの感染者数"] = all_jp["各地の感染者数合計"] / (all_jp["人口"] / 100000)
all_jp["直近1週間の10万人当たりの感染者数"] = all_jp["直近1週間の感染者数"] / (all_jp["人口"] / 100000)
all_jp.sort_values("都道府県コード")

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
4,北海道,502,37959,1112,3012.0,-0.031250,-0.014804,5267762,72.059064,720.590642,57.177982
41,青森県,1004,2352,30,174.0,10.000000,1.430487,1275783,18.435737,184.357371,13.638683
19,岩手県,1506,1442,45,90.0,-0.333333,0.041044,1235517,11.671228,116.712275,7.284400
11,宮城県,2008,8854,83,117.0,-0.600000,0.034394,2292385,38.623530,386.235296,5.103855
36,秋田県,2510,755,14,13.0,NaN,NaN,985416,7.661739,76.617388,1.319240
15,山形県,3012,1961,41,65.0,-0.375000,-0.100015,1082296,18.118888,181.188880,6.005751
35,福島県,3514,4583,144,115.0,0.181818,0.026165,1881981,24.351999,243.519993,6.110582
38,茨城県,4016,9734,149,307.0,-0.428571,0.019416,2921436,33.319231,333.192307,10.508531
27,栃木県,4518,6342,78,234.0,-0.500000,-0.019918,1965516,32.266336,322.663362,11.905271
37,群馬県,5020,7822,138,220.0,-0.526316,0.010630,1969439,39.716894,397.168940,11.170694


## 感染者数

### 平均値、中央値

In [12]:
all_jp["各地の感染者数合計"].describe()

count        47.000000
mean      15821.638298
std       29126.538045
min         465.000000
25%        2427.500000
50%        4789.000000
75%       10211.500000
max      160955.000000
Name: 各地の感染者数合計, dtype: float64

### TOP 10

In [13]:
all_jp.sort_values("各地の感染者数合計", ascending=False).reset_index(drop=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
0,東京都,6526,160955,2055,3830.0,-0.419643,0.010371,13834925,116.339626,1163.396260,27.683562
1,大阪府,13554,99719,2315,1768.0,-0.502538,-0.063929,8849635,112.681483,1126.814835,19.978225
2,神奈川県,7028,61515,884,1542.0,-0.403433,-0.044476,9209442,66.795578,667.955778,16.743685
3,愛知県,11546,47291,791,2435.0,-0.524590,-0.046589,7575530,62.425995,624.259953,32.142966
4,埼玉県,5522,43774,784,871.0,-0.450820,0.033752,7390054,59.233667,592.336673,11.786111
5,兵庫県,14056,39705,1181,761.0,-0.712871,-0.039807,5549568,71.546110,715.461095,13.712779
6,北海道,502,37959,1112,3012.0,-0.031250,-0.014804,5267762,72.059064,720.590642,57.177982
7,千葉県,6024,37131,677,738.0,-0.043011,0.020302,6319772,58.753702,587.537019,11.677636
8,福岡県,20080,34104,457,1188.0,-0.565789,-0.110448,5129841,66.481593,664.815927,23.158613
9,沖縄県,23594,17044,150,1859.0,-0.476015,0.163332,1481547,115.041912,1150.419123,125.476951


## １万人あたりの感染者数

### 平均値、中央値

In [14]:
all_jp["１万人当たりの感染者数"].describe()

count     47.000000
mean      37.908718
std       26.960875
min        7.661739
25%       20.405569
50%       29.264377
75%       49.875893
max      116.339626
Name: １万人当たりの感染者数, dtype: float64

## 直近１週間の１０万人あたりの感染者数

In [15]:
all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
28,沖縄県,23594,17044,150,1859.0,-0.476015,0.163332,1481547,115.041912,1150.419123,125.476951
4,北海道,502,37959,1112,3012.0,-0.031250,-0.014804,5267762,72.059064,720.590642,57.177982
24,愛知県,11546,47291,791,2435.0,-0.524590,-0.046589,7575530,62.425995,624.259953,32.142966
21,広島県,17068,10689,131,820.0,-0.177215,-0.044028,2826858,37.812299,378.122990,29.007470
26,東京都,6526,160955,2055,3830.0,-0.419643,0.010371,13834925,116.339626,1163.396260,27.683562
44,高知県,19578,1468,20,187.0,-0.200000,0.109142,709230,20.698504,206.985040,26.366623
34,福岡県,20080,34104,457,1188.0,-0.565789,-0.110448,5129841,66.481593,664.815927,23.158613
31,石川県,8534,3752,105,257.0,0.785714,0.497740,1139612,32.923486,329.234862,22.551535
17,岐阜県,10542,8614,163,446.0,-0.222222,0.028015,2032490,42.381512,423.815123,21.943527
29,滋賀県,12550,5107,79,292.0,-0.205128,0.081609,1420948,35.940794,359.407945,20.549661


## 直近１週間の新規感染者 増加速度

## 増加速度 統計情報

In [16]:
all_jp["直近1週間の新規感染者数前日比平均"].describe()

count    43.000000
mean      0.124380
std       0.400646
min      -0.114286
25%      -0.044252
50%       0.020302
75%       0.094376
max       2.107153
Name: 直近1週間の新規感染者数前日比平均, dtype: float64

### WORST 10(増加速度の早い順)

In [17]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=False).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
6,和歌山県,15060,2618,44,51.0,-0.800000,2.107153,954258,27.434928,274.349285,5.344467
41,青森県,1004,2352,30,174.0,10.000000,1.430487,1275783,18.435737,184.357371,13.638683
12,宮崎県,22590,3025,25,56.0,3.000000,0.600113,1095903,27.602808,276.028079,5.109941
31,石川県,8534,3752,105,257.0,0.785714,0.497740,1139612,32.923486,329.234862,22.551535
0,三重県,12048,4913,106,198.0,0.157895,0.301794,1813859,27.085898,270.858981,10.915953
10,奈良県,14558,7767,115,150.0,0.000000,0.177677,1353837,57.370274,573.702743,11.079620
28,沖縄県,23594,17044,150,1859.0,-0.476015,0.163332,1481547,115.041912,1150.419123,125.476951
16,山梨県,9538,1550,19,70.0,0.400000,0.163149,826579,18.751989,187.519886,8.468640
39,長崎県,21084,2892,65,63.0,-0.769231,0.121743,1350769,21.410026,214.100264,4.664010
44,高知県,19578,1468,20,187.0,-0.200000,0.109142,709230,20.698504,206.985040,26.366623


### BEST 10(増加速度の遅い順／減少速度の速い順)

In [18]:
all_jp.sort_values("直近1週間の新規感染者数前日比平均",ascending=True).head(10)

,都道府県名,都道府県コード,各地の感染者数合計,各地の死者数合計,直近1週間の感染者数,新規感染者数前日比,直近1週間の新規感染者数前日比平均,人口,１万人当たりの感染者数,10万人当たりの感染者数,直近1週間の10万人当たりの感染者数
22,徳島県,18072,1631,61,8.0,-1.000000,-0.114286,742505,21.966182,219.661820,1.077434
34,福岡県,20080,34104,457,1188.0,-0.565789,-0.110448,5129841,66.481593,664.815927,23.158613
15,山形県,3012,1961,41,65.0,-0.375000,-0.100015,1082296,18.118888,181.188880,6.005751
25,新潟県,7530,3206,36,139.0,-0.578947,-0.080648,2236042,14.337834,143.378344,6.216341
8,大分県,22088,3369,47,174.0,-0.434783,-0.075592,1151229,29.264377,292.643775,15.114282
30,熊本県,21586,6249,97,267.0,-0.576923,-0.066496,1769880,35.307478,353.074785,15.085769
9,大阪府,13554,99719,2315,1768.0,-0.502538,-0.063929,8849635,112.681483,1126.814835,19.978225
13,富山県,8032,1869,35,112.0,0.000000,-0.052319,1055999,17.698880,176.988804,10.606071
24,愛知県,11546,47291,791,2435.0,-0.524590,-0.046589,7575530,62.425995,624.259953,32.142966
18,岡山県,16566,7340,105,369.0,-0.222222,-0.045428,1903627,38.557974,385.579738,19.384050


## グラフ化

In [19]:
import matplotlib.pyplot as plt
%matplotlib notebook
#%matplotlib inline

In [20]:
all_jp_desc = all_jp.sort_values(["各地の感染者数合計"],ascending=False)
all_jp_desc_10k = all_jp.sort_values(["１万人当たりの感染者数"],ascending=False)
all_jp_desc_100k = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"],ascending=False)

In [21]:
fig = plt.figure(figsize=(9.5,10))
plt.subplot(2,2,1) # (rows, columns, panel number)
#plt.xticks(rotation="70",fontsize=9)
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["各地の感染者数合計"])
#plt.bar(all_jp_desc["都道府県名"], all_jp_desc["各地の感染者数合計"])
plt.barh(chart_data["都道府県名"],chart_data["各地の感染者数合計"],align="center")
#plt.title("各地の感染者数", y=0.9)
plt.title("各地の感染者数({})".format(latest_date),y=0.95)
plt.text(5,-1,"※{}時点".format(latest_date),fontsize=9)

plt.subplot(2,2,2) # (rows, columns, panel number)
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["１万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["１万人当たりの感染者数"])
plt.title("1万人当たりの感染者数({})".format(latest_date),y=0.95)
plt.text(5,-1,"※{}時点".format(latest_date),fontsize=9)

plt.subplot(2,2,3) # (rows, columns, panel number)
plt.yticks(fontsize=8)
plt.grid(axis="x")
chart_data = all_jp.sort_values(["直近1週間の10万人当たりの感染者数"])
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の10万人当たりの感染者数"])
plt.title("直近1週間の10万人当たりの新規感染者数",y=0.95)
# plt.text(0,chart_data["都道府県名"][-1],"{}時点".format(latest_date))
plt.text(5,-1,"※{}時点".format(latest_date),fontsize=9)

### 新規感染者数の増加速度（直近１週間の前日比平均）
plt.subplot(2,2,4) # (rows, columns, panel number)
chart_data = all_jp.sort_values("直近1週間の新規感染者数前日比平均",na_position="first")
plt.yticks(fontsize=8)
plt.grid(axis="x")
plt.barh(chart_data["都道府県名"], chart_data["直近1週間の新規感染者数前日比平均"])
plt.title("新規感染者数の増加速度（直近１週間の前日比平均）",y=0.95)
plt.text(0,2,"※{}時点".format(latest_date),fontsize=9)

fig.tight_layout()


<IPython.core.display.Javascript object>

In [22]:
fig.savefig("out/covid-domestic.png")

## 度数分布表

In [23]:
import numpy as np
data = np.array(all_jp["各地の感染者数合計"])
 
# ヒストグラム
hist, bin_edges = np.histogram(data, bins=8)

print(hist)
print(bin_edges)

[38  4  2  1  1  0  0  1]
[   465.    20526.25  40587.5   60648.75  80710.   100771.25 120832.5
 140893.75 160955.  ]


## ヒストグラム

In [24]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["各地の感染者数合計"],bins=[0,20000,40000,80000,100000,120000,140000,160000])

<IPython.core.display.Javascript object>

(array([38.,  4.,  3.,  1.,  0.,  0.,  0.]),
 array([     0,  20000,  40000,  80000, 100000, 120000, 140000, 160000]),
 <a list of 7 Patch objects>)

In [25]:
fig = plt.figure(figsize=(10,3))
plt.hist(all_jp_desc["10万人当たりの感染者数"],bins=[0,200,400,600,800,1000,1200])


<IPython.core.display.Javascript object>

(array([11., 23.,  4.,  6.,  0.,  3.]),
 array([   0,  200,  400,  600,  800, 1000, 1200]),
 <a list of 6 Patch objects>)